In [2]:
from marked_words import marked_words
import pandas as pd
from collections import Counter

def pprint(dic):
    full_list = []
    for word in sorted(dic,key=lambda x: x[1],reverse=True):
#         print("%s, %.2f" % (word[0],word[1]))
        full_list.append(word[0])
    return full_list

In [13]:
df1=pd.read_csv('data/dv3/dv3_story_generations.csv')
df2= pd.read_csv('data/dv3_generations_nb_stories.csv')
df2=df2.drop(0)
# df2=df2.loc[(df2['prom/pt_num']==4)|(df2['prompt_num']==5)]
df2['race']=df2['race'].replace('a Latino', 'a Latine')

In [36]:
df=pd.concat([df1,df2])


df = df.loc[df['text'].notna()]
df.to_csv('data/dv3_story_generations.csv')

In [30]:
from marked_words import marked_words
import pandas as pd
from collections import Counter

def pprint(dic):
    full_list = []
    for word in sorted(dic,key=lambda x: x[1],reverse=True):
#         print("%s, %.2f" % (word[0],word[1]))
        full_list.append(word[0])
    return full_list


In [31]:
dv3_mw = {}
for race in df['race'].unique():
#     print('\n Top words for %s \n-------' % race)
    outs = pprint(marked_words(df, [race], ['race'],['a White']))
    dv3_mw[race] = outs
temps = []
for race in df['race'].unique():
#     print('\n Top words for %s \n-------' % race)
    temp = pprint(marked_words(df, ['a White'], ['race'],[race]))
    temps.extend(temp)
seen = Counter(temps).most_common()
dv3_mw['a White']=[w for w, c in seen if c == 4]


for race in df['gender'].unique():
#     print('\n Top words for %s \n-------' % race)
    outs = pprint(marked_words(df, [race], ['gender'],['M']))
    dv3_mw[race] = outs
temps = []
for race in df['gender'].unique():
#     print('\n Top words for %s \n-------' % race)
    temp = pprint(marked_words(df, ['M'], ['gender'],[race]))
    temps.extend(temp)

seen = Counter(temps).most_common()
dv3_mw['M']=[w for w, c in seen if c == 2]
    
    
# Top words for intersectional groups
for race in df['race'].unique():
    for gen in ['N','W']:
        print('\n Top words for %s %s \n-------' % (race,gen))
        dv3_mw[race+gen] = pprint(marked_words(df, [race, gen], ['race', 'gender'],['a White','M']))
        
#     dv3_mw[race] = outs


 Top words for a White N 
-------

 Top words for a White W 
-------

 Top words for a Black N 
-------

 Top words for a Black W 
-------

 Top words for an Asian N 
-------

 Top words for an Asian W 
-------

 Top words for a Middle-Eastern N 
-------

 Top words for a Middle-Eastern W 
-------

 Top words for a Latine N 
-------

 Top words for a Latine W 
-------


In [32]:
import numpy as np
import re
import sklearn.feature_selection 
from nltk.stem.porter import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC


vectorizer = CountVectorizer(binary = True, decode_error = u'ignore')
tokenizer = vectorizer.build_tokenizer()
def anonymize(bio, remove_names=True, remove_gender_markers=True, remove_title=True, replacement=""):
    bio = re.sub(r"\b(?:[Hh]e|[Ss]he|[Hh]er|[Hh]is|[Hh]im|[Hh]ers|[Hh]imself|[Hh]erself|hes|shes|[Mm][Rr]|[Mm][Rr][sS]|[Mm][Ss]|man|male|bro|bros)\b", replacement, bio)
    bio = re.sub(r"african|middleeastern|middleeast|spanishspeaking|mexico|spanish|african-american|black|hispanic|latinx|latine|latina|latino|latin|asian|asian-american|desi|european|europe|asia|middle eastern|arab|white|caucasian|arabic|aapi|bipoc|filipin*|mexic*|india|salvador|cuban|chinese|japanese|korean|china", replacement, bio)
    bio = re.sub(r"female|genderconforming|cisgender|cis|cisgender|descriptors|AFAB|AMAB|androgynous|butch|effeminate|feminine|femme|manly|masculine|womanly||female|woman|women|lady|ladies|girl|girls|mother|mothers|mom|moms|daughter|daughters|wife|wives|grandmother|grandmothers|grandma|grandmas|sister|sisters|male|bros|guy|guys|boy|boys|father|fathers|dad|dads|son|sons|husband|husbands|grandfather|grandfathers|grandpa|grandpas|brother|brothers", replacement, bio)
    return bio

alldata = df.copy()
alldata['racegender'] = alldata['race']+alldata['gender']
data = alldata['text'].str.lower().replace('[^\w\s]','',regex=True)
top_words = dict()
dv3_svm = {}
for st in ['race','gender','racegender']:
    print(st.upper())
    for d in data:
        try:
            anonymize(d)
        except TypeError:
            print(d)
    concept_data = [anonymize(d) for d in data]

    labels = alldata[st]

    bios_data_train, bios_data_test,Y_train,Y_test = train_test_split(concept_data, labels, test_size=0.2, random_state=42, stratify=labels)

    vectorizer = CountVectorizer(analyzer='word',min_df=0.001,binary=False)
    X_train = vectorizer.fit_transform(bios_data_train)
    X_test = vectorizer.transform(bios_data_test)
    accs = []
    feature_names = vectorizer.get_feature_names_out()
    for r in alldata[st].unique():
        svm = SVC(kernel='linear')
        Y_train_bin = Y_train==r
        svm.fit(X_train, Y_train_bin)
        acc=sklearn.metrics.accuracy_score(Y_test==r,svm.predict(X_test))
#         print("%s Accuracy: %.2f"%(r,acc))
        accs.append(acc)
        coef = svm.coef_.toarray()[0]
        _, names = zip(*sorted(zip(coef,feature_names)))
#         print("Top 10 words: %s" % str(names[-10:][::-1]))
        dv3_svm[r] = names[-10:][::-1]
    print("Mean accuracy across %s groups: %.2f ± %.2f"%(st,np.mean(accs),np.std(accs)))

RACE
Mean accuracy across race groups: 0.83 ± 0.06
GENDER
Mean accuracy across gender groups: 0.87 ± 0.08
RACEGENDER
Mean accuracy across racegender groups: 0.94 ± 0.01


In [33]:
rows=['a White',
 'a Black',
 'an Asian',
 'a Middle-Eastern',
 'a Latine',
 'M',
 'W','N',
 'a WhiteW',
 'a BlackW',
 'an AsianW',
 'a Middle-EasternW',
 'a LatineW','a WhiteN', 'a BlackN', 'an AsianN', 'a Middle-EasternN', 'a LatineN']

In [34]:
maps={
    
    'a White':'White', 'a Black':'Black', 'an Asian':'Asian', 'a Middle-Eastern':'ME', 'a Latine':'Latine', 'M':'\hline man', 'W':'woman', 'N':'nonbinary', 'a BlackW':'\\hline Black W', 'an AsianW':'Asian W', 'a Middle-EasternW':'ME W', 'a LatineW':'Latine W',
'a BlackN':'Black NB', 'an AsianN':'Asian NB', 'a Middle-EasternN':'ME NB', 'a LatineN':'Latine NB'
}

In [35]:
final_table = {}
flags=[]
for k in dv3_mw:
#     if ('N' not in k and 'WhiteW' not in k) or len(k)==1:
        boths = []
#         only = []
        for v in dv3_mw[k]:
            if v in dv3_svm[k]:
                tempword = '\\textit{%s}'%v
            else:
                tempword = v
            boths.append(tempword)
#             if v in dv3_mw[k]:
#                 boths.append(tempword)
#             else:
#                 only.append(tempword)
#         if len(boths)>= 20:
#             final_table[k]='\hl{%s}'%(', '.join(boths[:20]))
# #         print(k)
# #         print(len(boths))
#         if len(boths)<20:
#             max_length = 20
#             only_max = max_length - len(boths)
#             if len(boths)<only_max:
#                 num_needed = only_max-len(boths)
#                 flags.append((k,num_needed))
                
#             only = only[:min(len(only),only_max)]
        final_table[k]=', '.join(boths)

In [37]:
extras={}
for k in dv3_mw:
    print(k)
    svm_list= dv3_svm[k]
#     num_got = 0
    extra_words=[]
    for word in svm_list:
#         if num_got < num_n:
        if word not in final_table[k]:
#                 num_got +=1
#                 print(word)
            extra_words.append(word)
#     print(len(extra_words))
#     print(num_n)
    if len(extra_words)>0:
        extras[k] = '\\textcolor{gray}{(%s)}'%', '.join(extra_words)
    else:
        extras[k]=''
# print(extras)

a White
a Black
an Asian
a Middle-Eastern
a Latine
M
W
N
a WhiteN
a WhiteW
a BlackN
a BlackW
an AsianN
an AsianW
a Middle-EasternN
a Middle-EasternW
a LatineN
a LatineW


In [38]:
# maps = 
for k in rows:
    v = final_table[k]
# #     if k
#     if k in extras:
    try:
        print(maps[k]+' &'+v+', '+extras[k]+'\\\\')
    except KeyError:
        continue
# #     else:
#         try:
#     #             print(maps[k])
#             print(maps[k]+' &'+v+'\\\\')
#         except KeyError:
#             continue

White &white, john, megan, \textcolor{gray}{(sam, out, jack, group, town, mac, understood, over, lila, emi)}\\
Black &black, tyler, \textit{nathaniel}, ryder, \textcolor{gray}{(others, jane, nina, jeremiah, kiara, where, went, only, into)}\\
Asian &asian, i, \textit{ling}, \textit{mei}, \textit{li}, \textit{kai}, china, my, \textit{takashi}, beijing, martial, arts, \textit{hua}, shii, wei, shanghai, \textit{tomo}, \textcolor{gray}{(yujin, chen, city)}\\
ME &\textit{middle}, middleeastern, \textit{ali}, \textit{east}, \textit{hassan}, eastern, \textit{ahmed}, \textit{village}, \textit{farrah}, \textit{farid}, culture, saeed, fatima, desert, \textcolor{gray}{(began, country)}\\
Latine &latino, \textit{maria}, latina, \textit{juan}, mexico, hard, \textit{marisol}, \textit{veronica}, carlos, states, \textit{rafael}, worked, latin, mexican, \textit{determined}, her, jose, antonio, united, business, \textcolor{gray}{(identity, sole, josé, javier)}\\
\hline man &he, his, him, man, himself, \t